# DATATHON CAJAMAR: ATMIRA STOCK PREDICTION

##PREDICCION DE DATOS A ESTIMAR

### Equipo Enver

Universidad Carlos III de Madrid

2021

In [ ]:
#import os 
#os.chdir("drive/MyDrive/Datathon") #SE ASUME QUE EL WORKING DIRECTORY ES DONDE ESTA ESTE DIRECTORIO

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def preparar_datos(data,estimar=False,product_ids=None): #procesamiento

  new_data = data.copy()
  new_data['dia_atipico'] = new_data['dia_atipico'].astype('category')
  new_data['categoria_uno'] = new_data['categoria_uno'].astype('category')
  new_data['estado'] = new_data['estado'].astype('category')

  #ordenar por id y fecha
  new_data = new_data.sort_values(['id','fecha'])

  #eliminar categoria_dos, estado y antiguedad
  new_data.drop(["categoria_dos",'estado','antiguedad'],axis=1,inplace=True)

  if estimar: #si es el dataset a estimar haremos que unidades_vendidas sea N/A 
    new_data['unidades_vendidas'] = np.nan

  new_data = new_data.set_index(["id","fecha"])[['unidades_vendidas','visitas','precio','campaña']].unstack(level=-1) 

  #Si un producto no aparece en un periodo de tiempo se imputa como 0 para unidades_vendidas, visitas y campaña y como el ultimo precio que tenia para precio
  new_data['unidades_vendidas'] = new_data['unidades_vendidas'].fillna(0) if not estimar else np.nan
  new_data['visitas'] = new_data['visitas'].fillna(0)
  new_data['campaña'] = new_data['campaña'].fillna(0)
  new_data['precio'] = new_data['precio'].T.fillna(method='ffill').fillna(method='bfill').T

  if product_ids is None:
    product_ids = data.id.sort_values().unique()
  
  new_data = new_data.loc[product_ids,:]

  return new_data

In [ ]:
## cargar datos
estimar_df = pd.read_table("raw_data/Estimar2.txt", sep="|", decimal=',', na_values=['-'])
modelar_df = pd.read_table("raw_data/Modelar_UH2021.txt", sep="|", decimal=',', na_values=['-'])#.sample(frac=0.1) #small sample for EDA
target_col = "unidades_vendidas"

## preprocesamiento

#cambiar fecha a tipo datetime
modelar_df['fecha'] = pd.to_datetime(modelar_df['fecha'],format="%d/%m/%Y %H:%M:%S")
estimar_df['fecha'] = pd.to_datetime(estimar_df['fecha'],format="%Y-%m-%d")

#ordenar estimar y modelar por fecha e id
estimar_df = estimar_df.sort_values(['id','fecha'])
modelar_df = modelar_df.sort_values(['id','fecha'])

#eliminar filas duplicadas de modelar
modelar_df.drop_duplicates(subset=['fecha','id'],inplace=True)

#el dataset a estimar no contiene valores con estado=Rotura asi que eliminaremos esos del modelar
modelar_df = modelar_df[modelar_df.estado!='Rotura']

#tambien hay ids del modelar que no existen en el estimar, como no proveen informacion extra los quitaremos de aqui
modelar_df = modelar_df[modelar_df.id.isin(estimar_df.id)] #eliminar ids que no esten en el conjunto a estimar

#imputar valores no existentes en precio por sus mas cercanos en el pasado en modelar
id_dfs = [modelar_df[modelar_df.id==id_].fillna(method='ffill').fillna(method='bfill') for id_ in modelar_df.id.unique()]
modelar_df = pd.concat(id_dfs,axis=0)

In [ ]:
#preparar datos para el modelado
modelar_processed = preparar_datos(modelar_df)
estimar_processed = preparar_datos(estimar_df,estimar=True)

#combinar estos datos por fecha
df = pd.concat([modelar_processed,estimar_processed],axis=1)

In [ ]:
#dataset de categorias por producto
categorias_df = modelar_df[['id','categoria_uno']].drop_duplicates()

#dataset de dias atipicos
dias_atipicos = pd.concat((estimar_df[['fecha','dia_atipico']],modelar_df[['fecha','dia_atipico']]),axis=0).drop_duplicates()

Lo siguiente es una funcion pipeline que dado un dataframe con una fila para cada uno de los $p$ productos, un dia `d` y numero entero `days_to_pred` devuelve un dataframe nuevo con : 

1. Nuevas variables basadas en los valores futuros a `d` de `visitas`, `campaña` y `precio` por cada uno de los `days_to_pred` dias a predecir. 
2. Nuevas variables sacando la media de variables como `visitas`, `unidades_vendidas` y `precio` de $n$ dias pasados a `d` para varias $n$. 
3. Nuevas variables basadas en pasados dias de campaña (Como el numero de veces que el producto ha estado en campaña en los ultimos $n$ dias para varias $n$) y tambien basado en futuros dias de campañá (Si/no habran dias de campaña en el futuro cercano por ejemplo).
4. Variables dummies para las variables categoricas como `dia_atipico` y `categoria_uno`.

Por esto el dataset de entrada a esta pipeline es indexado por el `id` del producto en sus filas y por `fecha` en sus columnas como hemos hecho arriba.

In [ ]:
def transform_pipeline(df,d,df_after = None,days_to_pred=30): #funcion de pipeline
  '''
  Recibe un dataset df y dia d y genera nuevas variables temporales basada en los dias `days_to_pred` a predecir
  '''
  if df_after is None:
    df_after = df#.loc[:,(np.unique([r[0] for r in df.columns]),pd.date_range(d, periods=days_to_pred, freq='d') )]
  def get_values_on_period(df,days_before,days_length):
    date_range = pd.date_range(d-pd.to_timedelta(days_before,'days'),d-pd.to_timedelta(days_before,'days')+pd.to_timedelta(days_length-1,'days'))
    date_range = date_range[~date_range.astype(str).str.contains('02-29')]
    return df[date_range]
  #1. Informacion de los dias a predecir
  X_list = [
       pd.DataFrame({
            f'visitas_{i}_despues' : df_after['visitas'][d+pd.to_timedelta(i,'days')].values.ravel(),
            f'precio_{i}_despues' : df_after['precio'][d+pd.to_timedelta(i,'days')].values.ravel(),
            f'campana_{i}_despues' : df_after['campaña'][d+pd.to_timedelta(i,'days')].values.ravel(),
            f'demanda_alta_{i}_despues': (dias_atipicos[dias_atipicos.fecha==d+pd.to_timedelta(i,'days')].dia_atipico==1).astype(int).tolist()*len(df),
            f'demanda_baja_{i}_despues': (dias_atipicos[dias_atipicos.fecha==d+pd.to_timedelta(i,'days')].dia_atipico==-1).astype(int).tolist()*len(df),
            }
        ) for i in range(days_to_pred) if d+pd.to_timedelta(i,'days') in df_after['visitas'].columns
  ]
  #1. Informacion del pasado
  X_list = X_list + [
       pd.DataFrame({
            f'{var}_1_antes': get_values_on_period(df[var],1,1).values.ravel(),
            f'{var}_7_antes_avg': get_values_on_period(df[var],7,7).values.mean(axis=1),
            f'{var}_14_antes_avg': get_values_on_period(df[var],14,14).values.mean(axis=1),
            f'{var}_28_antes_avg': get_values_on_period(df[var],28,28).values.mean(axis=1),
            f'{var}_60_antes_avg': get_values_on_period(df[var],60,60).values.mean(axis=1),
            #f'{var}_120_antes_avg': get_values_on_period(df[var],120,120).values.mean(axis=1),
            }
        ) for var in ['unidades_vendidas','precio','visitas']
  ]
  #2.
  X_list = X_list + [ #conteo de cuantas veces el producto ha estado en campaña en dias pasados
       pd.DataFrame({
               f'count_campana_{i}_antes': get_values_on_period(df['campaña'],i,i).values.sum(axis=1) for i in [1,7,14,30]
               })
       ]
  X_list = X_list + [ #variables binaricas indicando si el producto estara en campaña cada dia de las proximas dos semanas
       pd.DataFrame({
               f'is_campana_{i}_despues': get_values_on_period(df['campaña'],i,i).values.sum(axis=1) for i in range(14)
               })
       ]
  #3.
  X_list = X_list + [ #dummies de categoria_uno
       pd.DataFrame({
               f'categoria_uno_{cat}': df.index.isin(categorias_df[categorias_df.categoria_uno==cat].id).astype(int) for cat in categorias_df.categoria_uno
               })
       ]
  X_list = X_list + [ #dummies de dia_atipico
       pd.DataFrame({
           f'demanda_alta_1_antes': [get_values_on_period(df['visitas'],1,1).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==1].fecha).sum()]*len(df),
           f'conteo_demanda_alta_7_antes': [get_values_on_period(df['visitas'],7,7).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==1].fecha).sum()]*len(df),
           f'conteo_demanda_alta_15_antes': [get_values_on_period(df['visitas'],15,15).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==1].fecha).sum()]*len(df),
           f'conteo_demanda_alta_30_antes': [get_values_on_period(df['visitas'],30,30).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==1].fecha).sum()]*len(df),
           f'conteo_demanda_alta_60_antes': [get_values_on_period(df['visitas'],60,60).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==1].fecha).sum()]*len(df),
           f'demanda_baja_1_antes': [get_values_on_period(df['visitas'],1,1).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==-1].fecha).sum()]*len(df),
           f'conteo_demanda_baja_7_antes': [get_values_on_period(df['visitas'],7,7).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==-1].fecha).sum()]*len(df),
           f'conteo_demanda_baja_15_antes': [get_values_on_period(df['visitas'],15,15).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==-1].fecha).sum()]*len(df),
           f'conteo_demanda_baja_30_antes': [get_values_on_period(df['visitas'],30,30).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==-1].fecha).sum()]*len(df),
           f'conteo_demanda_baja_60_antes': [get_values_on_period(df['visitas'],60,60).columns.isin(dias_atipicos[dias_atipicos.dia_atipico==-1].fecha).sum()]*len(df),
           })
       ]
  X = pd.concat(X_list,axis=1)
  
  return X

La funcion de pipeline definida arriba sera usada para obtener los conjuntos $X$ e $Y$ de entrenamiento que definiran respectivamente las variables que seran usadas para predecir y el target de cada modelo. Esta funcion obtiene un numero $m$ de variables temporales de hasta los ultimos $n$ dias del dataframe dado. Para tomar ventaja de los datos que nos han dado para modelar (que tienen un periodo de un año y medio) tomaremos $b$ "batches" de estos datos, cada uno con un periodo de alrededor de 3 meses, y por cada batch obtendremos $p$ nuevas filas para usar en el train. 

El dataframe que usaremos para entrenar tendra por lo tanto $b\cdot p$ filas y $m$ columnas.

In [ ]:
#Obtener conjuntos X e Y que seran usados para entrenar y estimar

b = 8 #numero de"batches"
batch_spacing = 45 #Numero de dias que separan un batch del otro

from fastprogress import progress_bar

estimar_from_date = estimar_df.fecha.iloc[0] #dia donde empieza la estimacion
n_days_to_pred = estimar_df.fecha.nunique() #numero de dias que predecir

X_estimar = transform_pipeline(df,estimar_from_date,days_to_pred=n_days_to_pred)
Y_hat_estimar = pd.DataFrame(data=None,index=df.index,columns=estimar_processed['unidades_vendidas'].columns) #dataset vacio que sera llenado luego cuando se saquen las predicciones

estimar_begin = df['precio'][pd.date_range(estimar_from_date, periods=n_days_to_pred, freq='d').difference([pd.to_datetime("2016-02-29")])].columns[0] #solo para asegurarse de que la fecha donde se empieza a estimar esta en df
estimar_end = df['precio'][pd.date_range(estimar_from_date, periods=n_days_to_pred, freq='d').difference([pd.to_datetime("2016-02-29")])].columns[-1]

assert estimar_end >= estimar_df.fecha.iloc[-1], "No estas estimando todo el conjunto a estimar"
assert estimar_begin <= estimar_df.fecha.iloc[0], "No estas estimando desde el inicio del conjunto a estimar"

vars = ['unidades_vendidas','visitas','precio','campaña']
train_end = estimar_begin - pd.to_timedelta(1,unit='days')
df_train = df.loc[:,(vars,pd.date_range(start=modelar_df.fecha.iloc[0], end=train_end ))]

assert train_end - pd.to_timedelta(batch_spacing*(b-1)+n_days_to_pred+61,'days')  > modelar_df.fecha.iloc[0], "La fecha de train_begin debe ser posterior a la primera fecha disponible en el conjunto de modelar. Reduce b o batch_spacing"

X_list = []
Y_list = []
train_batch_end = train_end-pd.to_timedelta(n_days_to_pred,unit='days') 
print("OBTENIENDO CONJUNTO DE ENTRENAMIENTO...")
for i in progress_bar(range(b)):
  train_batch_start = train_batch_end - pd.to_timedelta(61,'days') 
  day_target_start = train_batch_end+pd.to_timedelta(1,unit='days')

  train_period = pd.date_range(start=train_batch_start,end=train_batch_end)
  target_period = pd.date_range(day_target_start, periods=n_days_to_pred, freq='d')
  target_period = target_period if "2016-02-29" not in target_period else pd.date_range(day_target_start, periods=n_days_to_pred+1, freq='d').difference([pd.to_datetime("2016-02-29")])

  batch_df = df.loc[:,(vars,train_period)]
  days_after_df = df.loc[:,(('visitas','precio','campaña'),target_period)]

  X_batch = transform_pipeline(batch_df,day_target_start,days_after_df,n_days_to_pred)#.add_suffix(f"_b{i+1}")  
  Y_batch = df['unidades_vendidas'][target_period]
  Y_batch.columns = list(range(1,n_days_to_pred+1))

  X_list.append(X_batch); Y_list.append(Y_batch);   
  train_batch_end = train_batch_end - pd.to_timedelta(batch_spacing,unit='days')

X_train = pd.concat(X_list)
y_train = pd.concat(Y_list)

print("\nFecha donde empieza el conjunto a entrenar:",str(train_batch_start)) 
print("Fecha donde termina el conjunto a entrenar:",str(train_end)) 
print("Tamaño del conjunto a entrenar transformado:",X_train.shape)

print("\nFecha donde empieza el conjunto a estimar:",str(estimar_begin))
print("Fecha donde termina el conjunto a estimar:",str(estimar_end))
print("Tamaño del conjunto a estimar transformado:",X_estimar.shape)

OBTENIENDO CONJUNTO DE ENTRENAMIENTO...



Fecha donde empieza el conjunto a entrenar: 2015-06-20 00:00:00
Fecha donde termina el conjunto a entrenar: 2016-09-30 00:00:00
Tamaño del conjunto a entrenar transformado: (21888, 516)

Fecha donde empieza el conjunto a estimar: 2016-10-01 00:00:00
Fecha donde termina el conjunto a estimar: 2016-12-31 00:00:00
Tamaño del conjunto a estimar transformado: (2736, 516)


In [ ]:
## Metrica para early stopping:
from sklearn.metrics import mean_squared_error
rRMSE = lambda y_true,y_pred: mean_squared_error(y_true,y_pred,squared=False)/y_true.mean()
def my_score_function(y_true, y_pred):
  CF = sum(y_pred>=y_true)/len(y_true)
  rRMSE = mean_squared_error(y_true,y_pred,squared=False)/y_true.mean()
  return 0.7*rRMSE+(0.3*(1-CF))

my_score_fun = lambda y_hat, y: ('my_score_fun',my_score_function(y.get_label(),y_hat))

In [ ]:
import xgboost as xgb
xgb_params = {
    "objective": "reg:squarederror",
    'n_estimators':400,
    "eta": 0.07,
    "subsample": 0.8,
    'max_depth':6,
    'verbosity':0,
    'silent':0,
}

def train_xgb(X_train,Y_train,X_test,Y_test,params,feval):
  dtrain = xgb.DMatrix(X_train,Y_train)
  dtest = xgb.DMatrix(X_test,Y_test) if X_test is not None else None
  watchlist = [(dtrain, 'train'), (dtest, 'test')] if dtest is not None else [(dtrain, 'train')]
  gbm = xgb.train(params, dtrain, num_boost_round=20, evals = watchlist, early_stopping_rounds = 5, feval = feval,verbose_eval=0)
  if dtest is not None:
    pred = gbm.predict(dtest)
    eval_metric = my_score_function(Y_test,pred)
  else:
    eval_metric = np.nan
  return (eval_metric,gbm)

print("ENTRENANDO CADA UNO DE LOS %s MODELOS..."%n_days_to_pred)
results = {}
for j in progress_bar(range(n_days_to_pred)):
  #print(f"{i+1}/{train_X.id.nunique()},{id_}")
  y_tr_j = y_train.iloc[:,j]
  results[j+1] = train_xgb(X_train,y_tr_j,None,None,xgb_params,my_score_fun)

ENTRENANDO CADA UNO DE LOS 92 MODELOS...


In [ ]:
#predicciones:
print("OBTENIENDO PREDICCIONES...")
D_estimar = xgb.DMatrix(X_estimar)
for j in progress_bar(range(n_days_to_pred)):
  yhat = results[j+1][1].predict(D_estimar)
  Y_hat_estimar.iloc[:,j] = yhat

OBTENIENDO PREDICCIONES...


In [ ]:
#cambiar al formato adecuado para la entrega:
entrega = pd.DataFrame(data=None,columns=['FECHA','ID','UNIDADES'])
print("OBTENIENDO DATASET DE ENTREGA...")
for id in progress_bar(estimar_df.id.unique()):
  id_fechas = estimar_df[estimar_df.id==id].fecha
  df = pd.DataFrame({'FECHA':id_fechas,'ID':[id]*len(id_fechas),'UNIDADES':Y_hat_estimar.loc[id,id_fechas].values})
  entrega = entrega.append(df)

assert all(entrega.FECHA.isin(estimar_df.fecha)) and all(estimar_df.fecha.isin(entrega.FECHA)) and all(entrega.ID.isin(estimar_df.id)) and all(estimar_df.id.isin(entrega.ID)), "Hay algo mal con el dataset de entrega!"

#post-procesamiento
entrega.UNIDADES = np.ceil(np.where(entrega.UNIDADES<1,0,entrega.UNIDADES)).astype(int)
entrega.FECHA = entrega.FECHA.dt.strftime("%d/%m/%Y")
entrega

OBTENIENDO DATASET DE ENTREGA...


,FECHA,ID,UNIDADES
0,01/10/2016,21972,2
1,02/10/2016,21972,3
2,03/10/2016,21972,2
3,04/10/2016,21972,2
4,05/10/2016,21972,2
...,...,...,...
212836,26/12/2016,458660,35
212837,27/12/2016,458660,46
212838,28/12/2016,458660,40
212839,29/12/2016,458660,63


In [ ]:
#Exportar como archivo de texto con el formato adecuado:
entrega.to_csv("Enver.txt",sep="|",decimal=",",index=False)